In [ ]:
%matplotlib widget
# Documentation needed for ipyml - notifications on push from github

from bmcs_utils.api import InteractiveModel, InteractiveWindow, Item, View
from bmcs_beam.models.moment_curvature.moment_curvature_ import MomentCurvature

import numpy as np
import sympy as sp
import traits.api as tr
import matplotlib.pylab as plt

from matplotlib.path import Path
import matplotlib.patches as mpatches
from matplotlib.patches import PathPatch
import ipywidgets as widgets

from scipy.integrate import cumtrapz
sp.init_printing()

from scipy.signal import find_peaks
from sympy.physics.continuum_mechanics.beam import Beam
# https://docs.sympy.org/latest/modules/physics/continuum_mechanics/beam_problems.html


In [ ]:
class CrackOrientationSymbolic(SymbExpr):
    """This class handles all the symbolic calculations
    so that the class crack orientation doesn't use sympy ever
    """
    name = 'CrackOrientationSymbolic'

    #Far field Stress
    bar_sigma = 1
    eta = tr.Float(0)

    ipw_view = View(
        Item('eta', param=True, latex='\eta', minmax=(0,1)),
    )

    #-------------------------------------------------------------------------
    # Symbolic derivation of expressions
    #-------------------------------------------------------------------------

    a = sp.Symbol('a', nonnegative = True)
    r = sp.Symbol('r', nonnegative = True)
    s_b, c_b = sp.symbols('s_b, c_b')
    s_t, c_t = sp.symbols('s_t,c_t')
    beta = sp.Symbol(r'\beta', real = True)
    theta = sp.Symbol(r'\theta', real = True)

    #-------------------------------------------------------------------------
    # Symbolic derivation of expressions
    #-------------------------------------------------------------------------

    # Far field stress rotation along the crack and stress intensity factors
    #-------------------------------------------------------------------------
    sigma_ij = sp.Matrix([[(1 - self.eta) * self.bar_sigma, self.eta * self.bar_sigma],
                    [ self.eta * self.bar_sigma, 0]])

    R_beta = sp.Matrix([[c_b, -s_b],
                    [s_b, c_b]])
    sigma_beta_ij = R_beta.T * sigma_ij * R_beta
    sigma_beta_ij_sim_ = sp.simplify(sigma_beta_ij)

    K_I = sp.sqrt(sp.pi * a) * sigma_beta_ij_sim_[0,0]
    K_II  = sp.sqrt(sp.pi * a) * sigma_beta_ij_sim_[1,0]

    # Stress at crack tip
    #-------------------------------------------------------------------------

    #Angular functions
    s_3t = 3 * s_t - 4 * s_t**3
    c_3t = 4 * c_t**3 - 3 * c_t
    S_I_11 = 2 * c_t - c_t * (1 + s_t * s_3t)
    S_I_22 = c_t * (1 + s_t * s_3t)
    S_I_12 = c_t * s_t * c_3t
    S_I = sp.Matrix([[S_I_11, S_I_12],[S_I_12, S_I_22]])
    S_I_ = sp.simplify(S_I)

    S_II_11 = - 2 * s_t - s_t * c_t * c_3t
    S_II_22 = s_t * c_t * c_3t
    S_II_12 = c_t * (1 - s_t * s_3t)
    S_II = sp.Matrix([[S_II_11, S_II_12],[S_II_12, S_II_22]])
    S_II_ = sp.simplify(S_II)

    c_t_h = sp.sqrt((1 + c_t)/2)
    s_t_h = sp.sqrt((1 - c_t)/2)

    sigma_tip_ij = sp.Matrix([[K_I * S_I_[0,0] + K_II * S_II_[0,0], K_I * S_I_[0,1] + K_II * S_II_[0,1]],
                   [K_I * S_I_[0,1] + K_II * S_II_[0,1], K_I * S_I_[1,1] + K_II * S_II_[1,1]]])
    sigma_tip_ij_ = sp.simplify(sigma_tip_ij.subs(c_t, c_t_h).subs(s_t, s_t_h)) / sp.sqrt(2 * sp.pi * r)

    cs_theta = sp.Matrix([c_t, s_t])
    R_theta = sp.Matrix([[cs_theta[0], -cs_theta[1]],[cs_theta[1], cs_theta[0]]])

    sig_tip_theta_ij = R_theta.T * sigma_tip_ij_ * R_theta
    sig_tip_theta_ij_ = sp.simplify(sig_tip_theta_ij)
    s_t_ = sp.sqrt(1 - c_t**2)
    sig_tip_theta_ij_sub = sig_tip_theta_ij_.subs(s_t, s_t_)
    sig_tip_theta_ij_sim = sp.simplify(sig_tip_theta_ij_sub)

    # Maximum principal stress condition
    #-------------------------------------------------------------------------

    cos_theta_sigma_tension = sp.solve(sig_tip_theta_ij_sim[1,0], c_t)
    cos_theta = cos_theta_sigma_tension[2].subs(c_b, sp.cos(beta)).subs(s_b, sp.sin(beta))
    theta = sp.acos(cos_theta)
    get_theta_sig_pt = sp.lambdify(theta, 'numpy')

class CrackOrientation(InteractiveModel,InjectSymbExpr):
   """Class returning the crack orientation angles.
    """
    symb_class = CrackOrientationSymbolic
    
    #Far field Stress
    bar_sigma = 1
    eta = tr.Float(0)
    ipw_view = View(
        Item('eta', param=True, latex='\eta', minmax=(0,1)),
        Item('beta', param=True, minmax=(-1.57,1.57)
    )
    def get_theta(self, beta):
        '''Far Field Condition'''
        a = sp.Symbol('a', nonnegative = True)
        r = sp.Symbol('r', nonnegative = True)
        s_b, c_b = sp.symbols('s_b, c_b')
        s_t, c_t = sp.symbols('s_t,c_t')
        sigma_ij = sp.Matrix([[(1 - self.eta) * self.bar_sigma, self.eta * self.bar_sigma],
                    [ self.eta * self.bar_sigma, 0]])
        
        R_beta = sp.Matrix([[c_b, -s_b],
                    [s_b, c_b]])
        sigma_beta_ij = R_beta.T * sigma_ij * R_beta
        sigma_beta_ij_sim_ = sp.simplify(sigma_beta_ij)
        K_I = sp.sqrt(sp.pi * a) * sigma_beta_ij_sim_[0,0]
        K_II  = sp.sqrt(sp.pi * a) * sigma_beta_ij_sim_[1,0]
    
        #Angular functions
        s_3t = 3 * s_t - 4 * s_t**3
        c_3t = 4 * c_t**3 - 3 * c_t
        S_I_11 = 2 * c_t - c_t * (1 + s_t * s_3t) 
        S_I_22 = c_t * (1 + s_t * s_3t)
        S_I_12 = c_t * s_t * c_3t
        S_I = sp.Matrix([[S_I_11, S_I_12],[S_I_12, S_I_22]])
        S_I_ = sp.simplify(S_I)
        
        S_II_11 = - 2 * s_t - s_t * c_t * c_3t  
        S_II_22 = s_t * c_t * c_3t
        S_II_12 = c_t * (1 - s_t * s_3t)
        S_II = sp.Matrix([[S_II_11, S_II_12],[S_II_12, S_II_22]])
        S_II_ = sp.simplify(S_II)
        
        c_t_h = sp.sqrt((1 + c_t)/2)
        s_t_h = sp.sqrt((1 - c_t)/2)
        
        sigma_tip_ij = sp.Matrix([[K_I * S_I_[0,0] + K_II * S_II_[0,0], K_I * S_I_[0,1] + K_II * S_II_[0,1]],
                   [K_I * S_I_[0,1] + K_II * S_II_[0,1], K_I * S_I_[1,1] + K_II * S_II_[1,1]]])
        sigma_tip_ij_ = sp.simplify(sigma_tip_ij.subs(c_t, c_t_h).subs(s_t, s_t_h)) / sp.sqrt(2 * sp.pi * r)
        
        cs_theta = sp.Matrix([c_t, s_t])
        R_theta = sp.Matrix([[cs_theta[0], -cs_theta[1]],[cs_theta[1], cs_theta[0]]])
        
        sig_tip_theta_ij = R_theta.T * sigma_tip_ij_ * R_theta
        sig_tip_theta_ij_ = sp.simplify(sig_tip_theta_ij)
        s_t_ = sp.sqrt(1 - c_t**2)
        sig_tip_theta_ij_sub = sig_tip_theta_ij_.subs(s_t, s_t_)
        sig_tip_theta_ij_sim = sp.simplify(sig_tip_theta_ij_sub)

        cos_theta_sigma_tension = sp.solve(sig_tip_theta_ij_sim[1,0], c_t)
        cos_theta = cos_theta_sigma_tension[2].subs(c_b, sp.cos(beta)).subs(s_b, sp.sin(beta))
        theta = sp.acos(cos_theta)
        #get_theta_sig_pt = sp.lambdify(theta, 'numpy')
        return theta
    
    def subplots(self, fig):
        return fig.subplots(1, 1)

    def update_plot(self, ax):
        betas = np.linspace(0, 1.57, 100)
        thetas = np.zeros(100)
        for i, beta in enumerate(betas):
            thetas[i] = self.get_theta(beta)
        ax.plot(betas, thetas, linewidth=self.n_layers*self.thickness)

In [ ]:
crack_orientation = CrackOrientation()
print(crack_orientation.get_theta(1.57))
# interactive_window = InteractiveWindow(crack_orientation)
# interactive_window.interact()